In [1]:
import tensorflow as tf
print(tf.__version__)

2.5.0


In [2]:
# import tensorflow as tf
# from tensorflow.keras import layers, models
# from tensorflow.keras.applications.vgg16 import VGG16
# from sklearn.utils import shuffle
# from sklearn.model_selection import train_test_split

import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics

In [3]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [4]:
df = pd.read_csv("tracks.csv")
df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4


In [5]:
df['release_date']=pd.to_datetime(df['release_date']).dt.year
df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4


In [6]:
def get_compiled_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(14, activation='relu'),
        tf.keras.layers.Dense(28, activation='relu'),
        tf.keras.layers.Dense(28, activation='relu'),
        tf.keras.layers.Dense(14, activation='relu'),
        tf.keras.layers.Dense(1)
    ])

    model.compile(optimizer='adam',
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=['MeanAbsoluteError'])
    return model

In [7]:
feature = ['popularity', 'duration_ms', 'release_date', 'danceability', 'energy', 'key', 'loudness', 'mode', 
                'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
dic = {}
for i in feature:
    
    if i == 'popularity':
        continue
    
    feature_list = feature.copy()
    feature_list.remove(i)
    dft = df[feature_list]


    x = dft.iloc[:, 1:]  # The first to second-last columns are the features
    y = dft.iloc[:, 0]   # The last column is the ground-truth label


    X_train, X_test, y_train, y_test = train_test_split(
        x, y, test_size=0.20, random_state=42)

    sc = StandardScaler()
    sc.fit(X_train)
    X_train_std = sc.transform(X_train)
    X_test_std = sc.transform(X_test)

    dataset = tf.data.Dataset.from_tensor_slices((X_train_std, y_train))

# for feat, targ in dataset.take(5):
#     print ('Features: {}, Target: {}'.format(feat, targ))

    train_dataset = dataset.shuffle(len(df)).batch(128)

    model = get_compiled_model()
    model.fit(train_dataset, epochs=100)
    MAE = model.evaluate(X_test_std, y_test.values)
    dic[i] = MAE
    print(dic)
    tf.keras.backend.clear_session()
#     print('remove ' + i + ' : ' + str(MAE))

Epoch 1/100
3667/3667 [==============================] - 5s 1ms/step - loss: 222.8395 - mean_absolute_error: 11.3353
Epoch 2/100
3667/3667 [==============================] - 4s 1ms/step - loss: 193.0410 - mean_absolute_error: 10.5691
Epoch 3/100
3667/3667 [==============================] - 4s 1ms/step - loss: 189.0221 - mean_absolute_error: 10.4111
Epoch 4/100
3667/3667 [==============================] - 4s 1ms/step - loss: 187.6502 - mean_absolute_error: 10.3544
Epoch 5/100
3667/3667 [==============================] - 4s 1ms/step - loss: 186.9772 - mean_absolute_error: 10.3274
Epoch 6/100
3667/3667 [==============================] - 4s 1ms/step - loss: 186.3920 - mean_absolute_error: 10.3054
Epoch 7/100
3667/3667 [==============================] - 5s 1ms/step - loss: 185.9950 - mean_absolute_error: 10.2937
Epoch 8/100
3667/3667 [==============================] - 5s 1ms/step - loss: 185.5617 - mean_absolute_error: 10.2786
Epoch 9/100
3667/3667 [==============================] - 5s 1ms/

In [8]:
print(dic)

{'duration_ms': [180.82437133789062, 10.168510437011719], 'release_date': [220.64154052734375, 11.765953063964844], 'danceability': [177.90489196777344, 10.012735366821289], 'energy': [176.6324005126953, 9.980091094970703], 'key': [176.23751831054688, 9.983283996582031], 'loudness': [179.3360595703125, 10.022418975830078], 'mode': [176.09103393554688, 9.950258255004883], 'speechiness': [176.00442504882812, 9.994057655334473], 'acousticness': [178.74354553222656, 9.988442420959473], 'instrumentalness': [177.3617706298828, 10.003206253051758], 'liveness': [175.98574829101562, 9.972467422485352], 'valence': [177.50831604003906, 10.057517051696777], 'tempo': [177.66714477539062, 9.924416542053223], 'time_signature': [176.3569793701172, 9.974599838256836]}


In [14]:
for i in dic.keys():
    print('remove ' + str(i) + " : " + str(dic[i][1]))

remove duration_ms : 10.168510437011719
remove release_date : 11.765953063964844
remove danceability : 10.012735366821289
remove energy : 9.980091094970703
remove key : 9.983283996582031
remove loudness : 10.022418975830078
remove mode : 9.950258255004883
remove speechiness : 9.994057655334473
remove acousticness : 9.988442420959473
remove instrumentalness : 10.003206253051758
remove liveness : 9.972467422485352
remove valence : 10.057517051696777
remove tempo : 9.924416542053223
remove time_signature : 9.974599838256836


In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 14)                196       
_________________________________________________________________
dense_1 (Dense)              (None, 28)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 28)                812       
_________________________________________________________________
dense_3 (Dense)              (None, 14)                406       
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 15        
Total params: 1,849
Trainable params: 1,849
Non-trainable params: 0
_________________________________________________________________
